In [1]:
import requests
import bs4 as bs
import pandas as pd
import country_converter as coco

url = 'https://en.wikipedia.org/wiki/COVID-19_lockdowns'
response = requests.get(url)
soup = bs.BeautifulSoup(response.text, "html.parser")
table = soup.find('table', {'class':"wikitable"})
df = pd.read_html(str(table))
df=pd.DataFrame(df[0])
df

vteCOVID-19 pandemic lockdowns  \
                                   Country / territory   
                                   Country / territory   
0                                              Albania   
1                                              Algeria   
2                                              Algeria   
3                                            Argentina   
4                                            Argentina   
..                                                 ...   
236                                            Vietnam   
237                                            Vietnam   
238                                            Vietnam   
239                                           Zimbabwe   
240  Outbreak ongoing: Lockdown data as of 13 March...   

                                                        \
                                                 Place   
                                                 Place   
0                                              Albania   
1                                              Algiers   
2                                                Blida   
3                                 Greater Buenos Aires   
4                                  Rest of the country   
..                                                 ...   
236                                          Hai Duong   
237                                           Bac Ninh   
238                                          Bac Giang   
239                                           Zimbabwe   
240  Outbreak ongoing: Lockdown data as of 13 March...   

                                                        \
                                        First lockdown   
                                            Start date   
0                                       2020-03-13[77]   
1                                       2020-03-23[79]   
2                                       2020-03-23[79]   
3                                       2020-03-19[81]   
4                                       2020-03-19[81]   
..                                                 ...   
236                                                NaN   
237                                                NaN   
238                                                NaN   
239                                    2020-03-30[507]   
240  Outbreak ongoing: Lockdown data as of 13 March...   

                                                        \
                                                         
                                              End date   
0                                       2020-06-01[78]   
1                                       2020-05-14[80]   
2                                       2020-05-14[80]   
3                                       2020-11-08[82]   
4                                       2020-05-10[85]   
..                                                 ...   
236                                                NaN   
237                                                NaN   
238                                                NaN   
239                                    2020-05-02[508]   
240  Outbreak ongoing: Lockdown data as of 13 March...   

                                                        \
                                                         
                                         Length (days)   
0                                                   80   
1                                                   52   
2                                                   52   
3                                                  234   
4                                                   52   
..                                                 ...   
236                                                 21   
237                                                 21   
238                                                 21   
239                                                 33   
240  Outbreak ongo

In [15]:
df1 = df.copy()
df1.columns = df1.columns.droplevel([0])
country_codes = ["AE", "EG", "CA", "IQ", "CN", "IS", "DK", "MX", "NL", "QA", "SG", "US"]
country_names = list(map(lambda x: coco.convert(x, to='name_short'), country_codes))

idx = df1["Country / territory"].isin(country_names)
idx

,Country / territory
0,False
1,False
2,False
3,False
4,False
...,...
236,False
237,False
238,False
239,False


In [166]:
df2 = df1[idx.values]
df2.columns = df2.columns.droplevel(0)
df2 = df2.rename({'Country / territory' : 'Country'}, axis = 1)
cols = []
count = 1
for column in df2.columns:
    if column == 'Start date':
        cols.append(f'Start_date_{count}')
        count+=1
        continue
    elif column == 'End date':
        cols.append(f'End_date_{count-1}')
        
        continue
 
    cols.append(column)
df2.columns = cols
df2 = df2.drop(["Length (days)", "Total length (days)", "Level"], axis = 1).dropna(axis = 1, how = 'all')
df2 = df2.applymap(lambda x: str(x).split('[')[0])

corona_lockdowns = {}
for country in df2.Country.unique():
    dates = []
    for i in range(4):
        start = pd.to_datetime(df2[df2.Country == country][f"Start_date_{i+1}"]).min()
        end   = pd.to_datetime(df2[df2.Country == country][f"End_date_{i+1}"]).max()
        if type(start) is pd.Timestamp:
            date_range = pd.Series(pd.date_range(start, end)).dt.date
            dates.extend(date_range)
    corona_lockdowns[coco.convert(country, to ='ISO2')] = dates
corona_lockdowns

{'CA': [datetime.date(2020, 3, 17),
  datetime.date(2020, 3, 18),
  datetime.date(2020, 3, 19),
  datetime.date(2020, 3, 20),
  datetime.date(2020, 3, 21),
  datetime.date(2020, 3, 22),
  datetime.date(2020, 3, 23),
  datetime.date(2020, 3, 24),
  datetime.date(2020, 3, 25),
  datetime.date(2020, 3, 26),
  datetime.date(2020, 3, 27),
  datetime.date(2020, 3, 28),
  datetime.date(2020, 3, 29),
  datetime.date(2020, 3, 30),
  datetime.date(2020, 3, 31),
  datetime.date(2020, 4, 1),
  datetime.date(2020, 4, 2),
  datetime.date(2020, 4, 3),
  datetime.date(2020, 4, 4),
  datetime.date(2020, 4, 5),
  datetime.date(2020, 4, 6),
  datetime.date(2020, 4, 7),
  datetime.date(2020, 4, 8),
  datetime.date(2020, 4, 9),
  datetime.date(2020, 4, 10),
  datetime.date(2020, 4, 11),
  datetime.date(2020, 4, 12),
  datetime.date(2020, 4, 13),
  datetime.date(2020, 4, 14),
  datetime.date(2020, 4, 15),
  datetime.date(2020, 4, 16),
  datetime.date(2020, 4, 17),
  datetime.date(2020, 4, 18),
  datetime.da